In [1]:
import pandas as pd
import requests
import csv

In [2]:
symbol = 'BTCUSDT'
tick_interval = '5m'

In [3]:
def get_binance_current_time():
    current_time_url = 'https://api.binance.com/api/v3/time'
    return requests.get(current_time_url).json()['serverTime']

In [4]:
def get_candles(start='', symbol='BTCUSDT', tick_interval='1d', limit=1000):
    
    base_url = 'https://api.binance.com/'
    endpoint = 'api/v3/klines?'
    
    if start:
        query = 'symbol=' + symbol + '&interval=' + tick_interval + '&startTime=' + str(start) +'&limit='+str(limit)
    else:
        query = 'symbol=' + symbol + '&interval=' + tick_interval +'&limit='+str(limit)
        
    candles = requests.get(base_url + endpoint + query).json()
    
    return candles, candles[-1][6]  # return candles and last colse time in a tuple

In [5]:
def get_all_candles_from_start(symbol, tick_interval):  # devuelve una lista de velas, cada vela es una lista tb
    
    start=1502942400000  # 17 de agosto de 2017
    _, last_time = get_candles(start='', symbol=symbol , tick_interval=tick_interval, limit=1)
    
    candles = []
    while start < last_time:
        i_candles, next_hop = get_candles(start, symbol, tick_interval)
        candles = candles + i_candles
        start = next_hop
        
    return candles

In [6]:
candles = get_all_candles_from_start(symbol, tick_interval)
columns=['open_time','open', 'high', 'low','close','volume','close_time','quote','trades',
         'takers_buy_base','takers_buy_quote','ignore']
df = pd.DataFrame(candles, columns=columns)

In [7]:
df = df.sort_values('close_time')
df.drop_duplicates(keep='last')
df = df.astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294269 entries, 0 to 294268
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   open_time         294269 non-null  float64
 1   open              294269 non-null  float64
 2   high              294269 non-null  float64
 3   low               294269 non-null  float64
 4   close             294269 non-null  float64
 5   volume            294269 non-null  float64
 6   close_time        294269 non-null  float64
 7   quote             294269 non-null  float64
 8   trades            294269 non-null  float64
 9   takers_buy_base   294269 non-null  float64
 10  takers_buy_quote  294269 non-null  float64
 11  ignore            294269 non-null  float64
dtypes: float64(12)
memory usage: 29.2 MB


In [8]:
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
df['close_time'] = df['close_time'].dt.tz_localize('utc').dt.tz_convert('Europe/Madrid')
df = df.set_index('close_time')

In [9]:
df.drop('ignore', axis=1, inplace=True)

In [10]:
df.to_csv(f'{symbol}_{tick_interval}.csv',index=False,header=True, quoting=csv.QUOTE_ALL)